In [3]:
# -*- coding: utf-8 -*-
from __future__ import annotations
from typing import List, Dict, Any, Optional
import random
import itertools
import networkx as nx


class NarrativeMap:
    """
    Interaktywna mapa narracyjna oparta na grafie NetworkX.

    Buduje drzewo decyzyjne, w którym każdy węzeł reprezentuje zdarzenie opisane
    2–3 zdaniami, a każda krawędź ma etykietę wyboru oraz atrybuty "ryzyko" i "nagroda".

    :param themes: Lista wątków przewodnich (np. ["odkupienie", "tajemnica"]).
    :type themes: List[str]
    :param locations: Lista lokacji (np. ["opuszczony zamek", "mglista dolina"]).
    :type locations: List[str]
    :param characters: Lista postaci (np. ["Wędrowiec", "Cień"]).
    :type characters: List[str]
    :param depth: Głębokość drzewa decyzyjnego (liczba poziomów decyzji).
    :type depth: int
    :param branching: Rozgałęzienie na każdy węzeł (liczba wyborów/gałęzi).
    :type branching: int
    :param rng_seed: Ziarno generatora losowego (dla powtarzalności).
    :type rng_seed: Optional[int]
    """

    # Zestaw par i pojedynczych etykiet wyborów; pierwsza para jest wymagana (“zaufaj” vs “zdrada”)
    _CHOICE_PAIRS: List[List[str]] = [
        ["zaufaj", "zdrada"],  # wymagane w specyfikacji
        ["prawda", "kłamstwo"],
        ["walcz", "uciekaj"],
        ["sojusz", "sabotuj"],
        ["poświęcenie", "chciwość"],
        ["rytuał", "profanacja"],
        ["straż", "włam"],
        ["otwórz", "zapieczętuj"],
    ]

    def __init__(
        self,
        themes: List[str],
        locations: List[str],
        characters: List[str],
        depth: int = 3,
        branching: int = 2,
        rng_seed: Optional[int] = None,
    ) -> None:
        if branching < 2:
            raise ValueError("Parametr 'branching' musi być >= 2.")
        if depth < 1:
            raise ValueError("Parametr 'depth' musi być >= 1.")
        if not themes or not locations or not characters:
            raise ValueError("Wymagane są niepuste listy: themes, locations, characters.")

        self.themes = themes
        self.locations = locations
        self.characters = characters
        self.depth = depth
        self.branching = branching
        self.rng = random.Random(rng_seed)

        # Graf skierowany: węzły = zdarzenia, krawędzie = wybory
        self.graph: nx.DiGraph = nx.DiGraph()
        self.root_id = 0

        # Budowa drzewa przy inicjalizacji
        self._build_tree()

    # --------------------------------------------------------------------- #
    # Public API
    # --------------------------------------------------------------------- #
    def get_path_summary(self, path: List[str]) -> str:
        """
        Zwraca opis przebiegu danej ścieżki wraz z podsumowaniem ryzyka/nagrody.

        Ścieżka to lista etykiet wyborów od korzenia (np. ["zaufaj", "kłamstwo", "walcz"]).
        Metoda przechodzi po kolejnych krawędziach zgodnie z etykietami i zbiera
        opisy zdarzeń oraz sumuje ryzyko/nagrodę.

        :param path: Lista etykiet wyborów (kolejne decyzje od korzenia).
        :type path: List[str]
        :return: Sformatowany tekstowy raport ze ścieżki.
        :rtype: str
        :raises ValueError: Jeśli którakolwiek etykieta wyboru nie istnieje z bieżącego węzła.
        """
        current = self.root_id
        lines: List[str] = []

        # Start – opis zdarzenia w korzeniu
        start_desc = self.graph.nodes[current]["description"]
        lines.append(f"▶ START: {start_desc}")
        total_risk = 0
        total_reward = 0

        for step_idx, choice in enumerate(path, start=1):
            # Znajdź krawędź z dopasowaną etykietą 'choice'
            found_edge = None
            for _, child, data in self.graph.out_edges(current, data=True):
                if data.get("choice") == choice:
                    found_edge = (child, data)
                    break
            if found_edge is None:
                available = [d.get("choice") for _, __, d in self.graph.out_edges(current, data=True)]
                raise ValueError(
                    f"Brak gałęzi o etykiecie '{choice}' z węzła {current}. "
                    f"Dostępne wybory: {available}"
                )

            child_id, edata = found_edge
            r, w = int(edata["risk"]), int(edata["reward"])
            total_risk += r
            total_reward += w
            lines.append(f"• KROK {step_idx} – wybór: “{choice}” (ryzyko={r}, nagroda={w})")

            # Opis kolejnego zdarzenia
            event_desc = self.graph.nodes[child_id]["description"]
            lines.append(f"  ⤷ Zdarzenie: {event_desc}")

            current = child_id

        lines.append(f"\n=== PODSUMOWANIE ===")
        lines.append(f"Decyzje: {len(path)} | Suma ryzyka: {total_risk} | Suma nagrody: {total_reward}")
        lines.append(f"Węzeł końcowy: {current} (głębokość={self.graph.nodes[current]['depth']})")
        return "\n".join(lines)

    def to_networkx(self) -> nx.DiGraph:
        """
        Zwraca wewnętrzny graf NetworkX (DiGraph).

        :return: Graf mapy narracyjnej.
        :rtype: nx.DiGraph
        """
        return self.graph

    def available_choices(self, node_id: int) -> List[str]:
        """
        Zwraca listę etykiet dostępnych wyborów z danego węzła.

        :param node_id: Identyfikator węzła (zdarzenia).
        :type node_id: int
        :return: Lista etykiet wyborów (krawędzi wychodzących).
        :rtype: List[str]
        """
        return [edata["choice"] for _, __, edata in self.graph.out_edges(node_id, data=True)]

    # --------------------------------------------------------------------- #
    # Internal helpers
    # --------------------------------------------------------------------- #
    def _build_tree(self) -> None:
        """
        Buduje drzewo decyzyjne w grafie NetworkX.

        Tworzy węzeł korzenia oraz kolejne poziomy do zadanej głębokości.
        Każdy węzeł otrzymuje opis zdarzenia (2–3 zdania).
        Każda krawędź otrzymuje etykietę wyboru, ryzyko (1–10) i nagrodę (1–10).

        :return: None
        :rtype: None
        """
        # Korzeń
        self.graph.add_node(
            self.root_id,
            depth=0,
            description=self._generate_event_description(self.root_id, depth=0),
        )

        next_node_id = self.root_id + 1
        frontier = [(self.root_id, 0)]

        # Przygotuj cykliczne źródło par wyborów dla kolejnych poziomów
        # Poziom 0 (z korzenia) – wymuszamy parę ["zaufaj", "zdrada"]
        choice_cycle = itertools.cycle(self._CHOICE_PAIRS[1:])  # bez pierwszej, bo ją użyjemy na starcie

        while frontier:
            parent_id, d = frontier.pop(0)
            if d >= self.depth:
                continue

            # Wybory dostępne z tego węzła
            if d == 0:
                # Pierwszy poziom: “zaufaj” vs “zdrada”
                base_choices = self._CHOICE_PAIRS[0].copy()
            else:
                # Kolejne poziomy: dobieramy z puli
                base_choices = list(next(choice_cycle))

            # Jeśli branching > 2, rozwijamy wybory, dociągając z puli następne etykiety
            while len(base_choices) < self.branching:
                base_choices.extend(next(choice_cycle))

            # Przytnij do dokładnego branching
            choices = base_choices[: self.branching]

            for choice in choices:
                # Losowe ryzyko/nagroda 1–10
                risk = self.rng.randint(1, 10)
                reward = self.rng.randint(1, 10)

                child_id = next_node_id
                next_node_id += 1

                # Dodaj węzeł (zdarzenie) i krawędź (wybór)
                self.graph.add_node(
                    child_id,
                    depth=d + 1,
                    description=self._generate_event_description(child_id, depth=d + 1),
                )
                self.graph.add_edge(
                    parent_id,
                    child_id,
                    choice=choice,
                    risk=risk,
                    reward=reward,
                )

                # Kolejne poziomy, jeśli jeszcze nie osiągnęliśmy maksymalnej głębokości
                if d + 1 < self.depth:
                    frontier.append((child_id, d + 1))

    def _generate_event_description(self, node_id: int, depth: int) -> str:
        """
        Generuje opis zdarzenia (2–3 zdania) na podstawie motywów, lokacji i postaci.

        Opis jest deterministycznie różnicowany na bazie node_id i depth,
        ale może być zastąpiony wywołaniem LLM.

        :param node_id: Identyfikator węzła.
        :type node_id: int
        :param depth: Głębokość węzła w drzewie.
        :type depth: int
        :return: Tekst opisu zdarzenia (2–3 zdania).
        :rtype: str
        """
        th = self.themes[node_id % len(self.themes)]
        loc = self.locations[(node_id + depth) % len(self.locations)]
        ch = self.characters[(node_id * 7 + depth) % len(self.characters)]

        # Dwie lub trzy krótkie frazy, spójne stylistycznie do RPG
        sentences = [
            f"W {loc} {ch} wyczuwa motyw '{th}', który splata niewidzialne nici przeznaczenia.",
            f"Cień dawnych wyborów kładzie się na drodze, a znaki ostrzegają przed konsekwencjami kolejnego kroku.",
        ]
        if self.rng.random() < 0.5:
            sentences.append(
                "Echa szeptów podpowiadają, że nawet najmniejsza decyzja może odmienić los całej wyprawy."
            )

        # TODO: call GPT here (np. self.llm.generate(...))
        return " ".join(sentences)
    # --------------------------------------------------------------------- #


# ------------------------------------------------------------------------- #
# Przykładowe użycie (zgodnie ze specyfikacją)
# ------------------------------------------------------------------------- #
if __name__ == "__main__":
    themes = ["odkupienie", "tajemnica"]
    locations = ["opuszczony zamek", "mglista dolina"]
    characters = ["Wędrowiec", "Cień"]

    nm = NarrativeMap(
        themes=themes,
        locations=locations,
        characters=characters,
        depth=3,       # trzy poziomy decyzji
        branching=2,   # binarne drzewo (np. “zaufaj” vs “zdrada”, ...)
        rng_seed=42,   # powtarzalność
    )

    # Przykładowa ścieżka: pierwszy poziom zawsze ma “zaufaj”/“zdrada”
    sample_path = ["zaufaj", "kłamstwo", "poświęcenie"] # Changed 'walcz' to 'poświęcenie'
    print(nm.get_path_summary(sample_path))

    # Można też sprawdzić dostępne wybory z korzenia:
    # print("Dostępne wybory z korzenia:", nm.available_choices(nm.root_id))

▶ START: W opuszczony zamek Wędrowiec wyczuwa motyw 'odkupienie', który splata niewidzialne nici przeznaczenia. Cień dawnych wyborów kładzie się na drodze, a znaki ostrzegają przed konsekwencjami kolejnego kroku.
• KROK 1 – wybór: “zaufaj” (ryzyko=1, nagroda=5)
  ⤷ Zdarzenie: W opuszczony zamek Wędrowiec wyczuwa motyw 'tajemnica', który splata niewidzialne nici przeznaczenia. Cień dawnych wyborów kładzie się na drodze, a znaki ostrzegają przed konsekwencjami kolejnego kroku. Echa szeptów podpowiadają, że nawet najmniejsza decyzja może odmienić los całej wyprawy.
• KROK 2 – wybór: “kłamstwo” (ryzyko=1, nagroda=1)
  ⤷ Zdarzenie: W opuszczony zamek Wędrowiec wyczuwa motyw 'odkupienie', który splata niewidzialne nici przeznaczenia. Cień dawnych wyborów kładzie się na drodze, a znaki ostrzegają przed konsekwencjami kolejnego kroku. Echa szeptów podpowiadają, że nawet najmniejsza decyzja może odmienić los całej wyprawy.
• KROK 3 – wybór: “poświęcenie” (ryzyko=1, nagroda=3)
  ⤷ Zdarzenie: W o